# Lab 19: K-means clustering

In labs 7-18 we studied *supervised learning*, meaning our training data consisted of the features or independent variables (x), and also labels or the dependent variable (y).  In other words, our training data included the answers for what we were trying to predict.

In comparison, in *unsupervised learning*, we do not have labeled data (y), only the independent variables (x).  Unsupervised learning algorithms try to find internal structure in this data.

In [ ]:
import seaborn as sns
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

from sklearn.cluster import KMeans

from sklearn.metrics import confusion_matrix

%matplotlib inline

### Section 1: Loading, cleaning, and plotting the data

We will use the Palmer penguin data set from Lab 3.  Recall it contains information about three species of penguins found in the Palmer Archipelago in Antarctica.

The CSV file can be downloaded from [https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv](https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv) or it can be accessed directly through the Seaborn library using the following command.  This second option only works for new versions of Seaborn, and may not work if using Jupyter Hub on Lehman 360 (in which case use the link for the CSV file above)

Load the penguin data set into the variable `penguins`

Drop the rows with missing data.

Plot scatterplots of all pairs of quantitative variables, colored by the species of penguin. 

Hint:  See Lab 3 for how to do this in one line of code.

Notice that penguins from the same species have similar physical measurements, and so group together (*cluster*) in the scatter plots.  In some graphs the groups for two different species overlap, but in other graphs we can more clearly see the three different species.

But what if we didn't already know which penguins belonged to which species (or even that there are 3 species of penguins in the data).  Could the computer figure this out?

### Section 2: K-Means clustering

A *clustering* algorithm is a program in which the computer tries to divide the data into a predetermined number of groups, such that data points in a group are more closely related to the other data points in that group than to data points in another group. Usually the Euclidean distance between the features or independent variables (x) is used to measure closeness, as in the K-Nearest Neighbors algorithm.

Let's try clustering the penguin data using the k-means clustering algorithm in the scikit-learn package.

First, create a new DataFrame `x` containing only the quantitative columns (`bill_length_mm`, `bill_depth_mm`, `flipper_length_mm`, `body_mass_g`).

Next create a KMeans object, which is a variable storing information about this clustering process (similar to how we created k-nearest neighbors variables, decision tree variables, etc. in the previous labs). 

The parameter `n_clusters = 3` tells the function that we want to look for 3 clusters, or groups.

In [ ]:
kmeans = KMeans(n_clusters=3)

Next we fit the data to the model.  Because clustering is an unsupervised method, we do not use a `y` variable.

In [ ]:
kmeans.fit(x)

Now we predict which cluster each data point will be in according to our model, and store this in the variable `clusters`.

In [ ]:
clusters = kmeans.predict(x)

Display the variable `clusters`.

How many different numbers are there?  What do you think these numbers represent?

### Section 3: Evaluating the clusters

How well did the clustering algorithm work?

One way to evaluate it is visually, by coloring the data points on our scatter plots by the clusters found by the algorith, not by the actual species.

We can do this by creating a new column in our DataFrame `penguins` called `clusters` with the clusters found in the last section.

<details><summary>Answer:</summary>
    <code>
    penguins["clusters"] = clusters
    </code>
</details>

Again plot the scatterplots between all pairs of quantitative variables, but this time color the points by the cluster.

Compare these plots to the first set of scatterplots that are colored by the actual species.  Did the k-means clustering algorithm do a good job?  Why or why not?

A second method to evaluate how well the clustering worked is with a confusion matrix.  We first have to map the cluster numbers to the species.  We can do this with the `map` function from Lab 4.

In [ ]:
cluster_map = {"0":"Adelie", "1":"Chinstrap", "2":"Gentoo"}
penguins["mapped_clusters"] = penguins["clusters"].apply(str).map(cluster_map)
penguins.head()

Notice that we have an extra function `apply(str)`, which is needed to change the `clusters` function from type `int` to string before applying the map.  However, by doing the type change as we apply the map, we did not change the type of the `clusters` column permanently.

We'll now create the confusion matrix using the scikit learn function from previous lab.  Can you figure out how to do it?  The true values (the `species` column) should be the first parameter.

<details><summary>Answer:</summary>
    <code>
    confusion_matrix(penguins["species"], penguins["mapped_clusters"])
    </code>
</details>

To know for sure which row/column corresponds to which penguin species, we can pass the order into the function as a parameter:

In [ ]:
confusion_matrix(penguins["species"], penguins["mapped_clusters"], labels = ["Adelie","Chinstrap","Gentoo"])

Which penguin species was most likely to be grouped correctly?  Which penguin species was least likely to be grouped correctly?  Is this what you expect from the scatterplots?

### Section 4: Scaling the data

If we look at the scatterplots involving the body mass variable, we can see that the penguins were mainly divided into clusters by their body mass.  One cluster seems to be penguins with a body mass of roughly 4000 g or less, another cluster seems to be penguins with a body mass of roughly 4000-5000g, and the last cluster seems to be penguins with a body mass greater than 5000g.  We don't see a strong division like this for any of the other variables.

What's happening?  Remember in K-Nearest Neighbors that we had to scale our variables to all have values between 0 and 1, so that a large variable didn't dominate the distance calculation?  The same situation is happening here.  Because close data points are placed in the same cluster and because the body mass variable values are so much bigger than the values of the other variables, the body mass variable dominates the distance computations.  In other words, deciding whether two data points are close or not depends mainly on the two body masses.

Let's see if scaling our data gives better clustering results.  We will scale the `x` data in the same way as in Lab 13.

First create a scaler object.

<details><summary>Answer:</summary>
<code>
scaler = MinMaxScaler()
</code>
</details>

Next use the `fit_transform()` function on the `scaler` object to scale the `x` data.

<details><summary>Answer:</summary>
<code>
x_scaled = scaler.fit_transform(x)
</code>
</details>

Create and fit a `KMeans` clustering object for 3 clusters to the scaled data.

<details><summary>Answer:</summary>
<code>
kmeans_scaled = KMeans(n_clusters = 3)
kmeans_scaled.fit(x_scaled)</code>
</details>

Cluster the scaled x data, and store it as a new column in the `penguins` DataFrame.

<details><summary>Answer:</summary>
<code>
penguins["clusters_scaled"] = kmeans_scaled.predict(x_scaled)
</code>
</details>

Are the clusters the same for the scaled and original unscaled data?

### Section 5: Evaluating the clusters with scaling

Let's evaluate the new clusters.  Plot the scatterplots for each pair of quantitative variables colored by the clusters created from the scaled data.

How do these scatterplots compare to the ones colored by species?

As in section 3, let's map the clusters to the species names and compute the confusion matrix.  Note that the map may be different than in section 3.

<details><summary>Answer:</summary>
<code>
cluster_scaled_map = {"1":"Adelie", "2":"Chinstrap", "0":"Gentoo"}
penguins["mapped_clusters_scaled"] = penguins["clusters_scaled"].apply(str).map(cluster_scaled_map)

confusion_matrix(penguins["species"], penguins["mapped_clusters_scaled"], labels = ["Adelie", "Chinstrap", "Gentoo"])
</code>
</details>

How does this confusion matrix compare to the confusion matrix from clustering the unscaled data?  

Did we get better clustering results from using the scaled or unscaled data?